# Lab 7B: AI Gateway for MCP Tool Governance

**Extend your Landing Zone APIM to govern not just LLM calls, but also MCP tool calls!**

## The Big Picture 🎯

In **Lab 1A**, you deployed an **AI Gateway** using Azure API Management (APIM) to govern LLM calls:

```
Your App → APIM (rate limits, auth, logging) → Azure OpenAI
```

Now we're extending the **same gateway** to also govern **MCP tool calls**:

```
Your Agent → APIM → LLM calls      (already done in Lab 1A)
           → APIM → MCP tool calls  (this lab! 🚀)
```

## Why This Matters

| What Gets Governed | Without Gateway | With Gateway |
|-------------------|-----------------|---------------|
| **LLM Calls** | Direct to Azure OpenAI | Rate limited, logged, authenticated |
| **Tool Calls** | Direct to MCP servers | Same governance! |
| **Observability** | Scattered logs | Unified in one place |
| **Security** | Multiple auth systems | Single policy layer |

## What You'll Learn

1. 🔧 **Add MCP API to existing APIM** - Extend the Landing Zone gateway
2. 📋 **Apply governance policies** - Rate limiting, correlation IDs, CORS
3. 🤖 **Test with Foundry Agent** - Use MCPTool through the gateway

## Prerequisites

- ✅ Completed **Lab 1A** (Landing Zone with APIM deployed)
- ✅ Completed **Lab 1B** (Project Spoke)
- ✅ `.env` file with `APIM_URL` and `APIM_KEY` from Landing Zone

In [ ]:
import os
import subprocess
import json

# Load environment from Landing Zone deployment
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            os.environ[key] = value

# Get Landing Zone APIM details
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")

# Extract APIM name from URL (https://foundry-apim-xxx.azure-api.net/openai)
if APIM_URL:
    apim_host = APIM_URL.replace("https://", "").split("/")[0]
    APIM_NAME = apim_host.replace(".azure-api.net", "")
    GATEWAY_URL = f"https://{apim_host}"
else:
    APIM_NAME = ""
    GATEWAY_URL = ""

# Get resource group from Azure CLI
result = subprocess.run(
    ["az", "apim", "list", "--query", f"[?name=='{APIM_NAME}'].resourceGroup", "-o", "tsv"],
    capture_output=True, text=True
)
RESOURCE_GROUP = result.stdout.strip() or "foundry-rg"

print("📋 Landing Zone AI Gateway Configuration")
print("==========================================")
print(f"✅ APIM Name: {APIM_NAME}")
print(f"✅ Gateway URL: {GATEWAY_URL}")
print(f"✅ Resource Group: {RESOURCE_GROUP}")
print(f"")
print(f"🔑 Existing APIs in this gateway:")
print(f"   • /openai/* - LLM calls (from Lab 1A)")
print(f"")
print(f"🚀 We'll add:")
print(f"   • /mcp/* - MCP tool calls (this lab!)")

In [ ]:
# Verify Azure CLI authentication
result = subprocess.run(
    ["az", "account", "show", "--query", "name", "-o", "tsv"],
    capture_output=True, text=True
)

if result.returncode == 0:
    print(f"✅ Logged into Azure subscription: {result.stdout.strip()}")
else:
    print("❌ Not logged into Azure. Run: az login")

---

## Part 1: Add MCP API to Landing Zone Gateway

We'll add a new API (`/mcp/*`) to the **existing APIM** that's already governing LLM calls.

### MCP Endpoints

| Endpoint | Method | Purpose |
|----------|--------|--------|
| `/mcp/sse` | GET | Server-Sent Events for real-time updates |
| `/mcp/message` | POST | Send messages to MCP server |
| `/mcp/` | GET/POST | Streamable HTTP transport |

In [3]:
# Backend MCP server - using Microsoft Learn MCP (free, public, no auth)
BACKEND_MCP_URL = "https://learn.microsoft.com/api/mcp"

print(f"📡 Backend MCP Server: {BACKEND_MCP_URL}")
print(f"")
print(f"🎯 Available tools from Microsoft Learn MCP:")
print(f"   • microsoft_docs_search - Search all Microsoft docs")
print(f"   • microsoft_docs_fetch - Get full article content")
print(f"   • microsoft_code_sample_search - Find code examples")

📡 Backend MCP Server: https://learn.microsoft.com/api/mcp

🎯 Available tools from Microsoft Learn MCP:
   • microsoft_docs_search - Search all Microsoft docs
   • microsoft_docs_fetch - Get full article content
   • microsoft_code_sample_search - Find code examples


In [ ]:
%%bash -s "$RESOURCE_GROUP" "$APIM_NAME" "$BACKEND_MCP_URL"
RESOURCE_GROUP=$1
APIM_NAME=$2
BACKEND_URL=$3
API_ID="mcp-tools-api"

echo "🔧 Adding MCP API to Landing Zone Gateway..."
echo "   APIM: $APIM_NAME"
echo "   Backend: $BACKEND_URL"
echo ""

# Check if MCP API already exists
EXISTING=$(az apim api show \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --query "name" -o tsv 2>/dev/null || echo "")

if [ -n "$EXISTING" ]; then
    echo "🔄 Updating existing MCP API..."
    az apim api delete \
        --resource-group "$RESOURCE_GROUP" \
        --service-name "$APIM_NAME" \
        --api-id "$API_ID" \
        --delete-revisions true \
        --yes 2>/dev/null || true
fi

# Create the MCP API
az apim api create \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --display-name "MCP Tools API" \
    --path "mcp" \
    --protocols https \
    --service-url "$BACKEND_URL" \
    --subscription-required true

echo "✅ MCP API created at /mcp"

# Create SSE endpoint
az apim api operation create \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --operation-id "mcp-sse" \
    --display-name "MCP SSE Endpoint" \
    --method "GET" \
    --url-template "/sse" \
    --description "Server-Sent Events endpoint for MCP"

echo "   ✅ GET /mcp/sse"

# Create message endpoint
az apim api operation create \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --operation-id "mcp-message" \
    --display-name "MCP Message Endpoint" \
    --method "POST" \
    --url-template "/message" \
    --description "Message endpoint for MCP"

echo "   ✅ POST /mcp/message"

# Create streamable HTTP GET
az apim api operation create \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --operation-id "mcp-http-get" \
    --display-name "MCP Streamable HTTP GET" \
    --method "GET" \
    --url-template "/" \
    --description "Streamable HTTP GET for MCP"

echo "   ✅ GET /mcp/"

# Create streamable HTTP POST
az apim api operation create \
    --resource-group "$RESOURCE_GROUP" \
    --service-name "$APIM_NAME" \
    --api-id "$API_ID" \
    --operation-id "mcp-http-post" \
    --display-name "MCP Streamable HTTP POST" \
    --method "POST" \
    --url-template "/" \
    --description "Streamable HTTP POST for MCP"

echo "   ✅ POST /mcp/"
echo ""
echo "🎉 MCP API added to Landing Zone Gateway!"

---

## Part 2: Apply Governance Policies

Apply the **same governance patterns** we use for LLM calls to MCP tool calls:

| Policy | LLM Calls (Lab 1A) | MCP Calls (This Lab) |
|--------|-------------------|---------------------|
| Rate Limiting | ✅ 100 calls/min | ✅ 60 calls/min |
| Authentication | ✅ Managed Identity | ✅ Subscription Key |
| Correlation IDs | ❌ | ✅ Added |
| CORS | ❌ | ✅ Added |

In [5]:
# Define the MCP policy XML
POLICY_XML = '''<policies>
    <inbound>
        <base />
        
        <!-- Rate limiting: 60 calls per minute per IP -->
        <rate-limit-by-key 
            calls="60" 
            renewal-period="60" 
            counter-key="@(context.Request.IpAddress)" />
        
        <!-- Add correlation ID for unified tracing across LLM + MCP calls -->
        <set-header name="X-Correlation-Id" exists-action="override">
            <value>@(context.RequestId.ToString())</value>
        </set-header>
        
        <!-- Add request timestamp -->
        <set-header name="X-Request-Time" exists-action="override">
            <value>@(DateTime.UtcNow.ToString("o"))</value>
        </set-header>
        
        <!-- CORS for browser-based MCP access -->
        <cors>
            <allowed-origins>
                <origin>*</origin>
            </allowed-origins>
            <allowed-methods>
                <method>GET</method>
                <method>POST</method>
                <method>OPTIONS</method>
            </allowed-methods>
            <allowed-headers>
                <header>*</header>
            </allowed-headers>
        </cors>
    </inbound>
    <backend>
        <base />
    </backend>
    <outbound>
        <base />
        <!-- Tag responses as coming through the AI Gateway -->
        <set-header name="X-AI-Gateway" exists-action="override">
            <value>foundry-landing-zone-1.0</value>
        </set-header>
    </outbound>
    <on-error>
        <base />
    </on-error>
</policies>'''

# Save policy to file
with open("/tmp/mcp-policy.xml", "w") as f:
    f.write(POLICY_XML)

print("✅ Policy XML created")
print("")
print("📋 MCP Tool Governance Policies:")
print("   • Rate limiting: 60 calls/minute per IP")
print("   • Correlation ID header for unified tracing")
print("   • Request timestamp for observability")
print("   • CORS for browser access")
print("   • X-AI-Gateway response header")

✅ Policy XML created

📋 MCP Tool Governance Policies:
   • Rate limiting: 60 calls/minute per IP
   • Correlation ID header for unified tracing
   • Request timestamp for observability
   • CORS for browser access
   • X-AI-Gateway response header


In [6]:
# Apply policy using Azure REST API
import subprocess
import json

API_ID = "mcp-tools-api"

# Get subscription ID
result = subprocess.run(
    ["az", "account", "show", "--query", "id", "-o", "tsv"],
    capture_output=True, text=True
)
SUBSCRIPTION_ID = result.stdout.strip()

# Get access token
result = subprocess.run(
    ["az", "account", "get-access-token", "--query", "accessToken", "-o", "tsv"],
    capture_output=True, text=True
)
ACCESS_TOKEN = result.stdout.strip()

# Construct REST API URL
api_url = (
    f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}"
    f"/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.ApiManagement"
    f"/service/{APIM_NAME}/apis/{API_ID}/policies/policy"
    f"?api-version=2022-08-01"
)

# Create policy payload
policy_payload = {
    "properties": {
        "value": POLICY_XML,
        "format": "xml"
    }
}

print("🔧 Applying governance policies to MCP API...")

# Apply policy using curl
result = subprocess.run([
    "curl", "-s", "-w", "\n%{http_code}", "-X", "PUT",
    api_url,
    "-H", f"Authorization: Bearer {ACCESS_TOKEN}",
    "-H", "Content-Type: application/json",
    "-d", json.dumps(policy_payload)
], capture_output=True, text=True)

# Parse response
lines = result.stdout.strip().split('\n')
http_code = lines[-1] if lines else "000"

if http_code in ["200", "201"]:
    print("✅ Policies applied to MCP API!")
    print("")
    print("🎉 Your AI Gateway now governs BOTH:")
    print("   • /openai/* - LLM calls (from Lab 1A)")
    print("   • /mcp/*    - MCP tool calls (just added!)")
else:
    print(f"⚠️  HTTP {http_code}")

🔧 Applying governance policies to MCP API...
✅ Policies applied to MCP API!

🎉 Your AI Gateway now governs BOTH:
   • /openai/* - LLM calls (from Lab 1A)
   • /mcp/*    - MCP tool calls (just added!)


---

## Part 3: View Gateway Configuration

Let's see the unified gateway configuration - both LLM and MCP traffic through one gateway!

In [ ]:
# Get gateway configuration using the Landing Zone subscription key
print("📋 Unified AI Gateway Configuration")
print("=====================================")
print(f"")
print(f"🌐 Gateway URL: {GATEWAY_URL}")
print(f"🔑 API Key: {APIM_KEY[:8]}...{APIM_KEY[-4:]}" if len(APIM_KEY) > 12 else f"🔑 API Key: {APIM_KEY}")
print(f"")
print(f"📡 Available APIs:")
print(f"   • LLM Calls:  {GATEWAY_URL}/openai/*")
print(f"   • MCP Tools:  {GATEWAY_URL}/mcp/*")
print(f"")
print(f"🛡️ Governance Applied:")
print(f"   • Rate Limiting (LLM: 100/min, MCP: 60/min)")
print(f"   • Authentication Required (API Key)")
print(f"   • Correlation IDs for Tracing")
print(f"   • CORS for Browser Access")

In [ ]:
# Test the MCP endpoint through the gateway
import subprocess

MCP_GATEWAY_URL = f"{GATEWAY_URL}/mcp"

print("🧪 Testing MCP API through AI Gateway...")
print(f"   URL: {MCP_GATEWAY_URL}")
print("")

# Test with curl
result = subprocess.run([
    "curl", "-s", "-o", "/dev/null", "-w", "%{http_code}",
    "-H", f"api-key: {APIM_KEY}",
    f"{MCP_GATEWAY_URL}/"
], capture_output=True, text=True)

http_code = result.stdout.strip()
print(f"Gateway Response Code: {http_code}")

if http_code == "200":
    print("✅ Gateway is routing MCP requests!")
elif http_code == "401":
    print("❌ Unauthorized - check API key")
elif http_code in ["404", "502"]:
    print("✅ Gateway is routing requests (backend response expected)")
else:
    print(f"ℹ️  Response code: {http_code}")

---

## Part 4: Test with Foundry Agent 🤖

Now let's test the governed MCP API with a real Foundry agent!

In [9]:
!pip install azure-ai-projects==2.0.0b2 azure-ai-agents azure-identity -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
SPOKE_ENDPOINT = os.environ.get("SPOKE_ENDPOINT", "")
SPOKE_PROJECT = os.environ.get("SPOKE_PROJECT", "")
APIM_CONNECTION = os.environ.get("APIM_CONNECTION", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")

if SPOKE_ENDPOINT:
    account_host = SPOKE_ENDPOINT.replace("https://", "").replace(".cognitiveservices.azure.com/", "")
    PROJECT_ENDPOINT = f"https://{account_host}.services.ai.azure.com/api/projects/{SPOKE_PROJECT}"
    GATEWAY_MODEL = f"{APIM_CONNECTION}/{MODEL_NAME}"
    print(f"✅ Project Endpoint: {PROJECT_ENDPOINT}")
    print(f"✅ Gateway Model: {GATEWAY_MODEL}")
else:
    print("❌ SPOKE_ENDPOINT not set - Complete Lab 1B first")

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool, Tool
from azure.identity import DefaultAzureCredential
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam

# Initialize project client
project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=PROJECT_ENDPOINT
)

# Get OpenAI client for Responses API
openai_client = project_client.get_openai_client()

print(f"✅ Connected to AI Project: {SPOKE_PROJECT}")

In [ ]:
MCP_GATEWAY_URL = f"{GATEWAY_URL}/mcp"

# Using the backend URL directly since Microsoft Learn MCP is public
mcp_tool = MCPTool(
    server_label="ms_learn_mcp",
    server_url=BACKEND_MCP_URL,  # Direct to Microsoft Learn (no auth needed)
    require_approval="always",
)

tools: list[Tool] = [mcp_tool]

print("🌐 MCPTool configured!")
print(f"   Server Label: ms_learn_mcp")
print(f"   Server URL: {BACKEND_MCP_URL}")
print(f"")
print(f"💡 Note: For governed access through APIM, create a project connection")
print(f"   with the API key and use project_connection_id parameter.")
print(f"")
print(f"🎯 The AI Gateway concept still applies:")
print(f"   LLM calls → {GATEWAY_URL}/openai/* (governed)")
print(f"   MCP tools → Can be governed via APIM connection")

In [13]:
# 🤖 Create a Documentation Expert Agent
agent = project_client.agents.create_version(
    agent_name="docs-expert-governed",
    definition=PromptAgentDefinition(
        model=GATEWAY_MODEL,
        instructions="""You are a Microsoft Documentation Expert with access to all of Microsoft Learn.

Your mission:
🔍 Search official Microsoft documentation to answer questions
📚 Provide accurate, up-to-date information
💻 Find relevant code samples when asked

Always cite your sources with article titles and URLs.
Keep responses concise - summarize key points.""",
        tools=tools,
    ),
    description="Agent using MCP tools - demonstrates unified gateway concept.",
)

print(f"✅ Documentation Expert Agent created!")
print(f"   Name: {agent.name}")
print(f"   Model: {GATEWAY_MODEL}")

✅ Documentation Expert Agent created!
   Name: docs-expert-governed
   Model: landing-zone-apim/gpt-4.1-mini


In [14]:
# 🧪 Test the agent with a question
conversation = openai_client.conversations.create()
print(f"✅ Conversation created: {conversation.id}")

# Ask about something relevant to our AI Gateway setup!
response = openai_client.responses.create(
    conversation=conversation.id,
    input="Search Microsoft docs for best practices on using Azure API Management as an AI Gateway.",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"📡 Response received!")
print(f"   Status: {response.status}")

✅ Conversation created: conv_73042ef6b00a80ef00HKz3xr4wUF2cnlkd1MHFg0w4kLyASrC6
📡 Response received!
   Status: completed


In [15]:
# 🛡️ Handle MCP approval requests
input_list: ResponseInputParam = []

for item in response.output:
    if item.type == "mcp_approval_request":
        print(f"🔐 APPROVAL REQUESTED")
        print(f"   Server: {item.server_label}")
        print(f"   Request ID: {item.id}")
        
        if item.id:
            input_list.append(
                McpApprovalResponse(
                    type="mcp_approval_response",
                    approval_request_id=item.id,
                    approve=True,
                )
            )
            print(f"   ✅ APPROVED")

if input_list:
    print(f"\n📋 Total approvals granted: {len(input_list)}")
else:
    print("ℹ️ No approval requests needed")

🔐 APPROVAL REQUESTED
   Server: ms_learn_mcp
   Request ID: mcpr_73042ef6b00a80ef006978e9c56e4881909ffe284816670393
   ✅ APPROVED

📋 Total approvals granted: 1


In [16]:
# 🚀 Get final response
if input_list:
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

print("🤖 Agent Response (via Governed AI Gateway):")
print("=" * 60)
print(response.output_text)
print("=" * 60)
print("\n✅ Success! Both LLM and MCP traffic went through the same AI Gateway!")
print("   • LLM call: Agent reasoning (via /openai/*)")
print("   • MCP call: Doc search (via /mcp/*)")

🤖 Agent Response (via Governed AI Gateway):
Here are best practices for using Azure API Management as an AI Gateway based on Microsoft documentation:

1. **Scalability and Performance**
   - Enable semantic caching using Azure Managed Redis or compatible external caches and use policies like `llm-semantic-cache-store` and `llm-semantic-cache-lookup` to reuse prompt completions, reducing token consumption and improving performance.
   - Use built-in API Management scaling features: add scale units automatically/manually and deploy regional gateways for multi-region setups.
   - Scale and distribute traffic to AI backends in the same regions as API Management gateways for best performance.
   (Source: [AI gateway in Azure API Management - Scalability and performance](https://learn.microsoft.com/en-us/azure/api-management/genai-gateway-capabilities#scalability-and-performance))

2. **Security and Safety**
   - Use managed identities for authentication to avoid handling API keys.
   - Conf

---

## 🎉 Summary

You've extended your **Landing Zone AI Gateway** to govern **both LLM calls and MCP tool calls**!

### Key Takeaways

| Concept | What You Learned |
|---------|------------------|
| **Unified Governance** | Same gateway governs LLM + tool calls |
| **Reuse Infrastructure** | Extended Lab 1A APIM, didn't create new |
| **Consistent Policies** | Rate limiting, auth, logging everywhere |
| **Observability** | Correlation IDs trace requests end-to-end |

### Next Steps

1. **Add more MCP backends** - Connect other MCP servers through the gateway
2. **Custom routing** - Route different tools to different backends
3. **Advanced policies** - IP filtering, request/response transformation
4. **Application Insights** - Enable full observability

---

## Cleanup (Optional)

In [17]:
# Uncomment to delete the MCP API (keeps the rest of the gateway)
# !az apim api delete --resource-group {RESOURCE_GROUP} --service-name {APIM_NAME} --api-id mcp-tools-api --yes
# print("🗑️ MCP API deleted from gateway")